In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from adapters import *
from dataset import NumpyCsvDataSet
from runner import ExptRunner
from networks.autoencoder import AutoEncoder
from networks.composedautoencoder import ComposedAutoEncoder
from networks.DenseAutoEncoder import DenseAutoEncoder
from networks.ConvVae import ConvVae
from networks.vae import VAE
from networks.lossfunctions import *

In [ ]:
opt = np.get_printoptions()
np.set_printoptions(threshold=np.inf)

In [ ]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print (device)

In [ ]:
start = time.time()
data_folder = 'data/demoplanner/version2'
train_data = NumpyCsvDataSet(data_folder + '/train.csv', device=device)
test_data = NumpyCsvDataSet(data_folder + '/test.csv', device=device)
print ("Time Taken: {} sec".format(time.time() - start))

In [ ]:
print (train_data.data.shape)
print (test_data.data.shape)

print (train_data.data.device)
print (test_data.data.device)

In [ ]:
expts = [
#     ['DemoPl-V2-ComposedAutoEncoderNoMaxPool-MSE-', ComposedAutoEncoder, demopl_v1_data_adapter, mse_loss_adapter, demopl_v1_data_to_img],
#     ['DemoPl-V2-ComposedAutoEncoderNoMaxPool-SmoothL1-', ComposedAutoEncoder, demopl_v1_data_adapter, smooth_l1_loss_adapter, demopl_v1_data_to_img],
    ['DemoPl-V2-ComposedAutoEncoderWithMaxPool-MSE-', lambda: ComposedAutoEncoder(useMaxPool=True), demopl_v1_data_adapter, mse_loss_adapter, demopl_v1_data_to_img],
#     ['DemoPl-V2-ComposedAutoEncoderWithMaxPool-SmoothL1-', lambda: ComposedAutoEncoder(useMaxPool=True), demopl_v1_data_adapter, smooth_l1_loss_adapter, demopl_v1_data_to_img],
#     ['DemoPl-V2-DenseAutoEncoder3-MSE-', lambda: DenseAutoEncoder(enc_layer_dims=[1024, 128, 16]), demopl_v1_data_adapter, mse_loss_adapter, demopl_v1_data_to_img],
#     ['DemoPl-V2-DenseAutoEncoder3-SmoothL1-', lambda: DenseAutoEncoder(enc_layer_dims=[1024, 128, 16]), demopl_v1_data_adapter, smooth_l1_loss_adapter, demopl_v1_data_to_img],
#     ['DemoPl-V2-VAE-VAELoss-', VAE, demopl_v1_data_adapter, vae_loss_adapter, demopl_v1_data_to_img],
#     ['DemoPl-V2-ConvVAE-VAELoss-', ConvVae, demopl_v1_data_adapter, vae_loss_adapter, demopl_v1_data_to_img],
]

In [ ]:
train_epochs = 2
for i in range(len(expts)):
    inputs = expts[i]
    net = inputs[1]()
    net = net.to(device)
    runner = ExptRunner(inputs[0], net, train_data, test_data, inputs[2], inputs[3], data_to_img_func=inputs[4])
    print ("Log folder for expt {}: {}".format(i, runner.log_folder))
    runner.train(train_epochs, shouldShowReconstruction=True)
    runner.test()

In [ ]:
np.set_printoptions(**opt)